<a href="https://colab.research.google.com/github/abdulmajiid1/NLP-asisgnment-/blob/main/train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn pandas numpy joblib


In [ ]:
import pandas as pd

df = pd.read_csv("Kaggle_hate_speech.csv")

df.head()

,count,hate_speech_count,offensive_language_count,neither_count,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
# Rename 'tweet' column to 'text'
df = df.rename(columns={'tweet': 'text'})

# Make new label column: 1=foul/offensive, 0=proper
df['label'] = df['class'].apply(lambda x: 1 if x in [0, 1] else 0)

# Keep only text + label
df = df[['text', 'label']]
df.head()


,text,label
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [ ]:
import re

def clean_text(s):
    s = s.lower()
    s = re.sub(r"http\S+|www\S+", " ", s)   # remove links
    s = re.sub(r"@\w+", " ", s)             # remove @users
    s = re.sub(r"#\w+", " ", s)             # remove hashtags
    s = re.sub(r"[^a-z\s]", " ", s)         # keep only letters
    return s.strip()

df['text'] = df['text'].astype(str).map(clean_text)
df.head()


,text,label
0,rt as a woman you shouldn t complain about ...,0
1,rt boy dats cold tyga dwn bad for cuffin ...,1
2,rt dawg rt you ever fuck a bitch and ...,1
3,rt she look like a tranny,1
4,rt the shit you hear about me might be true...,1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['label'], test_size=0.2, random_state=42, stratify=df['label']
)

len(X_train), len(X_test)


(19826, 4957)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# TF-IDF (turn words into numbers)
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=3, max_df=0.9)

# Learn vocabulary and transform training data
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

# Train model
model = LogisticRegression(max_iter=1000, class_weight='balanced', solver='liblinear')
model.fit(X_train_vec, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000, solver='liblinear')

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test_vec)
print(classification_report(y_test, y_pred, digits=3))


              precision    recall  f1-score   support

           0      0.750     0.946     0.837       833
           1      0.988     0.936     0.962      4124

    accuracy                          0.938      4957
   macro avg      0.869     0.941     0.899      4957
weighted avg      0.948     0.938     0.941      4957



In [ ]:
import joblib
joblib.dump(model, "model.joblib")
joblib.dump(vectorizer, "vectorizer.joblib")
print("Model saved!")


Model saved!


In [ ]:
new_tweet = ["You are an idiot!"]
X_new = vectorizer.transform(new_tweet)
pred = model.predict(X_new)[0]
print("Prediction:", "Foul" if pred==1 else "Proper")


Prediction: Proper


In [ ]:
vectorizer.transform(["Have a nice day!"])


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 3 stored elements and shape (1, 17730)>